In [1]:
import findspark
findspark.init()

# Initialize SparkSession

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import explode
from pyspark.sql.functions import split

spark = SparkSession\
    .builder\
    .appName("StructuredNetworkWordCount")\
    .getOrCreate()

# Create Streaming Dataframe 


In [3]:
lines = spark \
    .readStream \
    .format("socket") \
    .option("host","localhost")\
    .option("port",9999)\
    .load()

words = lines.select(
    explode(
        split(lines.value, " ")
    ).alias("word")
)

wordCounts = words.groupBy("word").count()

In [4]:
print(wordCounts.printSchema())

root
 |-- word: string (nullable = false)
 |-- count: long (nullable = false)

None


**Running nc -lk 9999 in another UNIX terminal**

In [5]:
 # Start running the query that prints the running counts to the console
wordCounts \
    .writeStream \
    .outputMode("complete") \
    .format("console") \
    .start() \
    .awaitTermination()